In [1]:
from requests import get
from bs4 import BeautifulSoup as soup
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
url ='https://hoopshype.com/salaries/2018-2019/'

In [3]:
def hoops_hype(url):
    #function to parse each row
    def parse_row(row):
        return [x.text.strip() for x in row.findAll('td')]
    
    #Getting url and using Beautiful Soup to Parse it
    response = get(url) #get the url
    x = soup(response.content, 'html.parser') #Parse the url
    tables = x.find_all('table') #Find table
    rows = tables[0].find_all('tr') #Find rows in first table
    list_parsed_rows =[parse_row(row) for row in rows[0:]] #Parse out rows
    
    
    #Creating dataframe and modifying
    df = pd.DataFrame(list_parsed_rows[1:]) #make Dataframe
    df.columns = ["Rk","Team","Payroll","Adj_Payroll"]
    df['Year'] = list_parsed_rows[0][2]
    df.Payroll = [x.strip('$') for x in df.Payroll]
    df.Payroll = [x.replace(',','') for x in df.Payroll]
    df.Adj_Payroll = [x.strip('$') for x in df.Adj_Payroll]
    df.Adj_Payroll = [x.replace(',','') for x in df.Adj_Payroll]
    
    #Setting datatypes
    float_cols = ['Payroll','Adj_Payroll']
    int_cols = []
    
    df[float_cols]=df[float_cols].astype(float)
    df[int_cols]=df[int_cols].astype(int)
    
    # Creating Conference Labels
    Eastern =[
    'Atlanta','Boston','Chicago','Cleveland','Detroit','Indiana','Miami','Milwaukee',
    'Brooklyn','New York','Orlando','Philadelphia','Washington' ,'Toronto','Charlotte']
    Western = [
    'Houston','LA Clippers','LA Lakers','Minnesota' ,'Phoenix','Portland','Sacramento','San Antonio',
    'Oklahoma City','Utah' ,'Memphis' ,'New Orleans','Dallas' ,'Denver' ,'Golden State']
    
    East = df[df['Team'].isin(Eastern)]
    East['Conference'] = 'Eastern'
    West = df[df['Team'].isin(Western)]
    West['Conference'] = 'Western'
    
    df=pd.concat([West,East])
    df.sort_index(inplace=True)
 
    return df 
    
    

In [4]:
x = hoops_hype('https://hoopshype.com/salaries/2018-2019/')
x.head()

,Rk,Team,Payroll,Adj_Payroll,Year,Conference
0,1.,Miami,153171497.0,155696495.0,2018/19,Eastern
1,2.,Golden State,146291276.0,148702858.0,2018/19,Western
2,3.,Oklahoma City,144916427.0,147305340.0,2018/19,Western
3,4.,Toronto,137793831.0,140065330.0,2018/19,Eastern
4,5.,Milwaukee,130988604.0,133147922.0,2018/19,Eastern


### Looking at Syntax of the Hoops Hype  
  
https://hoopshype.com/salaries/  
https://hoopshype.com/salaries/2018-2019/  
https://hoopshype.com/salaries/2017-2018/  




In [5]:
def hoops_hype_year(start_year, end_year):
    for i in range(0,end_year-start_year):
        if i == 0:
            a = start_year+i
            b = start_year+i+1
            url=f'https://hoopshype.com/salaries/{a}-{b}/'
            df = hoops_hype(url)
            print(start_year)
        elif i > 0:
            a = start_year+i
            b= start_year+i+1
            url=f'https://hoopshype.com/salaries/{a}-{b}/'
            df2 = hoops_hype(url)
            df = df.append(df2)
            print(start_year+i)
    return df
  

In [13]:
df = hoops_hype_year(2013,2019)
df.tail()

2013
2014
2015
2016
2017
2018


,Rk,Team,Payroll,Adj_Payroll,Year,Conference
25,26.,Phoenix,108692835.0,110484604.0,2018/19,Western
26,27.,LA Lakers,107225482.0,108993069.0,2018/19,Western
27,28.,Sacramento,101466920.0,103139575.0,2018/19,Western
28,29.,Dallas,86958881.0,88392374.0,2018/19,Western
29,30.,Atlanta,79180081.0,80485339.0,2018/19,Eastern


In [7]:
len(df)
# df.head()

846

In [14]:
df.to_csv('NBA_payroll_2013to2019.csv')